# Recommender System

<h2>Content Based VS Collaborative Filetering</h2>

 Um dos métodos de desenvolvimento de um sistema de recomendação é o <b>content based filtering</b>. Ele se baseia em uma descrição do item, utilizando palavras chaves, assim como nas preferencias do usuário, para identificar o tipo de item que ele gosta. 
 Outro métodos para desenvolver é atraves do chamado <b>collaborative filtering</b>, este método se baseia na obtenção e analise de uma grande quantidade de informações sobre comportamentos do usuário, e prever o que pode interessar a ele através de similaridades entre usuários. Uma grande vantagem do <b>collaborative filtering</b> é conseguir boas recomendações sem depender de um conteudo analizado e entendimento sobre o item, por parte do sistema. 
 O <b>collaborative filtering</b> pode ser dividido em dois métodos, o <b>memory-based</b> e o <b>model-based</b>. O <b>model-based</b> utiliza a fatoração de matrizes, é um método de aprendizado não supervisionado para variaveis latententes e redução de dimensionalidade. A fatorização de matriz é muito usada em sistemas de recomendação por lidar melhor com escalabilidade e sparsity do que o <b>memory-based</b>. Nesta apresentação será implementado um <b>model-based collaborative filtering</b>, que utiliza o <b>SVD</b>.
 
<h2>SVD</h2>

 SVD é um tecnica de fatoração de matrizes comumente utilizada para produzir aproximações low-rank. Dada uma matriz m x n A, com rank r, o SVD será definido como $$SVD(A) = U\times S\times V^t$$ onde U, S e V tem dimensões m × m, m × n e n × n, respectivamente.
 A matriz S é uma matriz diagonal, tendo apenas r entradas não nulas, o que faz da dimensão das 3 matrizes m x r, r x r e r x n, respectivamente. As entradas da diagonal(s1, s2, s3, ..., sr) tem uma propriedadec onde si > 0 e s1 ≥ s2 ≥ ... ≥ sr.
 As primeiras r colunas de U e V representam os autovetores ortogonais, associados a r autovalores não nulos de $$AA^t$$ e $$A^tA$$, respectivamente.


![svd](img.jpg)


<h2>Implementação</h2>

 O dataset utilizado na implementação e teste é o MovieLens 100k disponivel em https://grouplens.org/datasets/movielens/100k/ , ele contem 100k classificações de filmes de 943 usuários numa seleção de 1682 filmes.
 

In [63]:
import numpy as np
import pandas as pd


header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=header)

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

Separamos os dados do data frame em treinamento e teste através da função cross_validation do SciKit-Learn, em seguida criamos matrizes com esses dados. Já que a sparsity dessas matrizes é muito grande, guardamos as matrizes no formato CSC, <b>Compressed Sparse Column</b>, para otimizar perfomance e uso de memória.

In [89]:
from sklearn import cross_validation
train_data, test_data = cross_validation.train_test_split(df, test_size=0.25)

from scipy.sparse import csc_matrix

train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = float(line[3])
train_data_matrix = csc_matrix(train_data_matrix, dtype=float)
    
test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = float(line[3])
test_data_matrix = csc_matrix(test_data_matrix, dtype=float)

print('Sparsity ' + str((float(n_users*n_items)-100000.0)*100/ float(n_users*n_items)) + '%')

Sparsity 93.69533063577546%


Calculamos o SVD de maneira optimizada utilizando o sparsesvd (https://pypi.python.org/pypi/sparsesvd/) e aplicamos o algoritimo encontrado no paper "Incremental Singular Value Decomposition Algorithms for Highly Scalable Recommender Systems" $$Pi,j =\bar{ri} + U\times \sqrt{S}^t(i)\times \sqrt{S}(j)\times V^t$$

In [156]:
import math
from sparsesvd import sparsesvd
from sklearn.metrics import mean_absolute_error


K = 10
U, s, Vt = sparsesvd(csc_matrix(train_data_matrix, dtype=float), K)

S = np.zeros((K, K), dtype=float)
S_no_sqrt = np.zeros((K, K), dtype=float)
for i in range(0, len(s)):
    S_no_sqrt[i,i] = s[i]
    S[i,i] = math.sqrt(s[i])
    
U = csc_matrix(np.transpose(U))
S = csc_matrix(S)
S_no_sqrt = csc_matrix(S_no_sqrt)
Vt = csc_matrix(Vt)

prediction = U * S_no_sqrt * Vt

erro_pred = prediction.toarray()[test_data_matrix.toarray().nonzero()]
erro_test = test_data_matrix.toarray()[test_data_matrix.toarray().nonzero()]

print('Simple SVD Recommender MAE: ' + str(mean_absolute_error(erro_pred, erro_test)))


# Paper's algorithm
StSVt = np.transpose(S) * S * Vt

estimated_ratings = np.zeros(shape=(n_users, n_items), dtype=float)
for row in range(n_users):
    pred = U[row, :] * StSVt
    estimated_ratings[row, : ] = np.mean(train_data_matrix[row]) + pred.todense()

erro_pred = estimated_ratings[test_data_matrix.toarray().nonzero()]
erro_test = test_data_matrix.toarray()[test_data_matrix.toarray().nonzero()]

print('Papers SVD Recommender MAE: ' + str(mean_absolute_error(erro_pred, erro_test)))

Simple SVD Recommender MAE: 2.41735172239
Papers SVD Recommender MAE: 2.14275511801
